In [1]:
# %% [markdown]
# # Minimal MCP-like Server & Client in Python
# 
# This notebook shows a *very small* MCP-style server/client pattern
# with three tools:
# 
# - `run_sql`: run a SQL query on a SQLite database under ./data
# - `validate_sql`: validate SQL syntax
# - `http_get`: provide simple internet access (HTTP GET)
# 
# The "protocol" is simplified: the client sends a JSON-like dict
# `{ "tool": "<name>", "args": { ... } }` to the server, and the server
# dispatches to the corresponding Python function.


In [2]:
# %% 
from __future__ import annotations

import sqlite3
from pathlib import Path
from dataclasses import dataclass
from typing import Any, Callable, Dict, Optional, List, Tuple
import json
import textwrap
import traceback

# We'll use the standard library for HTTP so there are no extra deps
import urllib.request
import urllib.error


In [3]:
# %% [markdown]
# ## 1. Minimal MCP-like abstractions: Tool, Server, Client
# 
# We define:
# 
# - `Tool`: wraps a callable with a name/description
# - `MCPServer`: registers tools and executes them on request
# - `MCPClient`: sends tool calls to the server


In [4]:
# %%
@dataclass
class Tool:
    name: str
    description: str
    func: Callable[[Dict[str, Any]], Any]

    def __call__(self, args: Dict[str, Any]) -> Any:
        return self.func(args)


class MCPServer:
    """
    Very small in-process 'server' that can:
      - register tools
      - handle incoming tool invocation requests
    """

    def __init__(self) -> None:
        self.tools: Dict[str, Tool] = {}

    def register_tool(self, tool: Tool) -> None:
        if tool.name in self.tools:
            raise ValueError(f"Tool {tool.name} already registered")
        self.tools[tool.name] = tool

    def handle_request(self, request: Dict[str, Any]) -> Dict[str, Any]:
        """
        Expected request format:
        {
            "id": "<optional id>",
            "tool": "<tool_name>",
            "args": { ... }
        }
        """
        req_id = request.get("id")
        tool_name = request.get("tool")
        args = request.get("args", {})

        if tool_name not in self.tools:
            return {
                "id": req_id,
                "ok": False,
                "error": f"Unknown tool: {tool_name}",
            }

        try:
            result = self.tools[tool_name](args)
            return {
                "id": req_id,
                "ok": True,
                "result": result,
            }
        except Exception as e:
            # Capture traceback for debugging
            tb = traceback.format_exc()
            return {
                "id": req_id,
                "ok": False,
                "error": str(e),
                "traceback": tb,
            }


class MCPClient:
    """
    Very small 'client' that sends requests to the server.
    In a real MCP setup, this would speak JSON-RPC over stdio or a socket.
    """

    def __init__(self, server: MCPServer) -> None:
        self.server = server
        self._next_id = 1

    def call_tool(self, tool_name: str, args: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
        if args is None:
            args = {}
        req_id = str(self._next_id)
        self._next_id += 1

        request = {
            "id": req_id,
            "tool": tool_name,
            "args": args,
        }
        response = self.server.handle_request(request)
        return response


In [5]:
# %% [markdown]
# ## 2. SQL Tools
# 
# Assumptions:
# 
# - You have a SQLite database file under the `data` folder.
# - You can change `DB_PATH` to point at the correct file.


In [6]:
# %%
# Path to your SQLite database under ./data
DB_PATH = Path("../data") / "Employee_Information.db"  # <-- change filename if needed
# print all the files under DB_PATH parent directory
print(list(DB_PATH.parent.glob("*")))


def _get_connection() -> sqlite3.Connection:
    if not DB_PATH.exists():
        raise FileNotFoundError(f"Database not found at {DB_PATH!r}")
    conn = sqlite3.connect(DB_PATH)
    return conn


[PosixPath('../data/create_db.ipynb'), PosixPath('../data/.DS_Store'), PosixPath('../data/data_information'), PosixPath('../data/data_documentation.txt'), PosixPath('../data/Employee_Information.db')]


In [7]:
# %% [markdown]
# ### 2.1 `run_sql` Tool
# 
# Executes a SQL query and returns:
# 
# - `columns`: list of column names
# - `rows`: list of rows (as lists)
# 
# *Note:* this is a simple demo; it does not enforce read-only queries.


In [8]:
# %%
def run_sql_tool(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Expected args:
    {
        "query": "<SQL string>",
        "params": [ ... ]  # optional
    }
    """
    query = args.get("query")
    if not isinstance(query, str):
        raise ValueError("run_sql: 'query' must be a string")

    params = args.get("params") or []

    with _get_connection() as conn:
        cur = conn.cursor()
        cur.execute(query, params)
        # Try to fetch results; some statements don't return rows
        try:
            rows = cur.fetchall()
        except sqlite3.ProgrammingError:
            rows = []

        columns = [desc[0] for desc in cur.description] if cur.description else []

    return {
        "query": query,
        "columns": columns,
        "rows": rows,
        "rowcount": len(rows),
    }


In [9]:
# %% [markdown]
# ### 2.2 `validate_sql` Tool
# 
# To validate SQL syntax, we:
# 
# - Open a SQLite connection
# - Run `EXPLAIN <query>` if possible, or just prepare the statement
# - Catch any `sqlite3.Error` as a syntax (or related) error


In [10]:
# %%
def validate_sql_tool(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Expected args:
    {
        "query": "<SQL string>"
    }
    """
    query = args.get("query")
    if not isinstance(query, str):
        raise ValueError("validate_sql: 'query' must be a string")

    # Strip trailing semicolons for EXPLAIN
    stripped = query.strip().rstrip(";")

    with _get_connection() as conn:
        cur = conn.cursor()
        try:
            # EXPLAIN is a convenient way to check syntax without running the query
            cur.execute(f"EXPLAIN {stripped}")
            # If that worked, we consider syntax OK (for SQLite)
            return {
                "query": query,
                "valid": True,
                "message": "SQL syntax appears valid (for SQLite).",
            }
        except sqlite3.Error as e:
            return {
                "query": query,
                "valid": False,
                "message": f"SQL error: {e}",
            }


In [11]:
# %% [markdown]
# ## 3. HTTP / Internet Access Tool
# 
# The `http_get` tool:
# 
# - Performs a simple HTTP GET request
# - Returns status, headers, and a truncated body


In [12]:
# %%
def http_get_tool(args: Dict[str, Any]) -> Dict[str, Any]:
    """
    Expected args:
    {
        "url": "<string>",
        "max_chars": 10000   # optional, default 5000
    }
    """
    url = args.get("url")
    if not isinstance(url, str):
        raise ValueError("http_get: 'url' must be a string")

    max_chars = args.get("max_chars") or 5000
    if not isinstance(max_chars, int) or max_chars <= 0:
        max_chars = 5000

    try:
        req = urllib.request.Request(url, method="GET")
        with urllib.request.urlopen(req) as resp:
            status = resp.status
            headers = dict(resp.headers.items())
            raw_bytes = resp.read()

        try:
            body = raw_bytes.decode("utf-8", errors="replace")
        except Exception:
            body = str(raw_bytes)

        truncated_body = body[:max_chars]
        truncated = len(body) > max_chars

        return {
            "url": url,
            "status": status,
            "headers": headers,
            "body": truncated_body,
            "truncated": truncated,
        }

    except urllib.error.HTTPError as e:
        return {
            "url": url,
            "status": e.code,
            "headers": dict(e.headers.items()) if e.headers else {},
            "body": f"HTTP error: {e.reason}",
            "truncated": False,
        }
    except urllib.error.URLError as e:
        return {
            "url": url,
            "status": None,
            "headers": {},
            "body": f"URL error: {e.reason}",
            "truncated": False,
        }


In [13]:
# %% [markdown]
# ## 4. Register Tools on the Server


In [14]:
# %%
server = MCPServer()

server.register_tool(
    Tool(
        name="run_sql",
        description="Run a SQL query against the SQLite database in ./data.",
        func=run_sql_tool,
    )
)

server.register_tool(
    Tool(
        name="validate_sql",
        description="Validate SQL syntax using SQLite.",
        func=validate_sql_tool,
    )
)

server.register_tool(
    Tool(
        name="http_get",
        description="Perform an HTTP GET request to provide internet access.",
        func=http_get_tool,
    )
)

client = MCPClient(server)

print("Registered tools:", list(server.tools.keys()))


Registered tools: ['run_sql', 'validate_sql', 'http_get']


In [15]:
# %% [markdown]
# ## 5. Example Usage (Client Side)
# 
# These calls show how an LLM (or any client) could invoke tools.


In [16]:
# %%
# Example: validate a SQL query
response = client.call_tool(
    "validate_sql",
    {"query": "SELECT name FROM sqlite_master WHERE type='table'"},
)
print(json.dumps(response, indent=2))


{
  "id": "1",
  "ok": true,
  "result": {
    "query": "SELECT name FROM sqlite_master WHERE type='table'",
    "valid": true,
    "message": "SQL syntax appears valid (for SQLite)."
  }
}


In [17]:
# %%
# Example: run a SQL query (make sure your DB and table exist)
# Adjust the query to match your actual schema.

example_query = "SELECT name FROM sqlite_master WHERE type='table'"

response = client.call_tool(
    "run_sql",
    {"query": example_query},
)

print(json.dumps(response, indent=2))


{
  "id": "2",
  "ok": true,
  "result": {
    "query": "SELECT name FROM sqlite_master WHERE type='table'",
    "columns": [
      "name"
    ],
    "rows": [
      [
        "countries"
      ],
      [
        "states"
      ],
      [
        "users"
      ],
      [
        "user_profiles"
      ],
      [
        "addresses"
      ],
      [
        "products"
      ],
      [
        "orders"
      ],
      [
        "order_items"
      ],
      [
        "payments"
      ],
      [
        "support_tickets"
      ]
    ],
    "rowcount": 10
  }
}


In [18]:
# %%
# Example: HTTP GET to provide internet access to the LLM
response = client.call_tool(
    "http_get",
    {"url": "https://httpbin.org/get", "max_chars": 1000},
)

print(json.dumps(response, indent=2)[:1000] + "...")


{
  "id": "3",
  "ok": true,
  "result": {
    "url": "https://httpbin.org/get",
    "status": 200,
    "headers": {
      "Date": "Sat, 15 Nov 2025 00:16:37 GMT",
      "Content-Type": "application/json",
      "Content-Length": "275",
      "Connection": "close",
      "Server": "gunicorn/19.9.0",
      "Access-Control-Allow-Origin": "*",
      "Access-Control-Allow-Credentials": "true"
    },
    "body": "{\n  \"args\": {}, \n  \"headers\": {\n    \"Accept-Encoding\": \"identity\", \n    \"Host\": \"httpbin.org\", \n    \"User-Agent\": \"Python-urllib/3.13\", \n    \"X-Amzn-Trace-Id\": \"Root=1-6917c665-7e5c3b7367279c730a50ebe4\"\n  }, \n  \"origin\": \"86.163.35.35\", \n  \"url\": \"https://httpbin.org/get\"\n}\n",
    "truncated": false
  }
}...


In [19]:
# %% [markdown]
# ## 6. (Optional) A Tiny "LLM Loop" Example
# 
# This is just to show how an LLM *could* choose tools based on user input.
# Here it's hard-coded logic; in a real system the LLM would decide which
# tool to call by itself.


In [20]:
# %%
def simple_llm_router(user_input: str) -> Dict[str, Any]:
    """
    Extremely naive "router" that decides which tool to call based on the user's message.
    In reality, the LLM would generate these tool calls.
    """
    user_input_lower = user_input.lower()

    if user_input_lower.startswith("run sql:"):
        query = user_input[len("run sql:"):].strip()
        return client.call_tool("run_sql", {"query": query})

    if user_input_lower.startswith("validate sql:"):
        query = user_input[len("validate sql:"):].strip()
        return client.call_tool("validate_sql", {"query": query})

    if user_input_lower.startswith("http get:"):
        url = user_input[len("http get:"):].strip()
        return client.call_tool("http_get", {"url": url})

    # Fallback: echo message
    return {
        "ok": True,
        "result": {
            "message": "No tool selected; echoing input.",
            "echo": user_input,
        },
    }

# Demo:
for msg in [
    "validate sql: SELECT 1",
    "run sql: SELECT name FROM sqlite_master WHERE type='table'",
    "http get: https://httpbin.org/get",
]:
    print(f"\n>>> {msg}")
    res = simple_llm_router(msg)
    print(json.dumps(res, indent=2)[:500] + "...")



>>> validate sql: SELECT 1
{
  "id": "4",
  "ok": true,
  "result": {
    "query": "SELECT 1",
    "valid": true,
    "message": "SQL syntax appears valid (for SQLite)."
  }
}...

>>> run sql: SELECT name FROM sqlite_master WHERE type='table'
{
  "id": "5",
  "ok": true,
  "result": {
    "query": "SELECT name FROM sqlite_master WHERE type='table'",
    "columns": [
      "name"
    ],
    "rows": [
      [
        "countries"
      ],
      [
        "states"
      ],
      [
        "users"
      ],
      [
        "user_profiles"
      ],
      [
        "addresses"
      ],
      [
        "products"
      ],
      [
        "orders"
      ],
      [
        "order_items"
      ],
      [
        "payments"
      ],
      [
     ...

>>> http get: https://httpbin.org/get
{
  "id": "6",
  "ok": true,
  "result": {
    "url": "https://httpbin.org/get",
    "status": 200,
    "headers": {
      "Date": "Sat, 15 Nov 2025 00:17:02 GMT",
      "Content-Type": "application/json",
      "